## Practical Exam: Insurance Upgrades
<p>Travel Assured provides travel services to its customers. They are based in the United States.</p>
<p>Travel Assured provides everything from flights and hotel bookings to holiday insurance.</p>
<p>The sales team wants to sell upgrades to customers. So they can do this, it is vital that the data is clean, accurate and available for reporting. </p>
<p>They need your help to prepare some data before they start to run a new promotion. </p>
<p>The data you need is in the database named <code>insurance</code>. </p>
<p><strong>Database Schema</strong></p>
<p><img src="https://assets.datacamp.com/production/project_1642/img/schema.png" alt="database schema"></p>
<h2 id="task1">Task 1</h2>
<p>The sales team want to use customer information to target their new promotion. But, they think the data may not be clean enough to use. The table below shows what the sales team expect the data types and format to be.</p>
<table>
<thead>
<tr>
<th>Column Name</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>customer_id</td>
<td>Unique integer (set by the database, can’t take any other value)</td>
</tr>
<tr>
<td>location</td>
<td>State names as a lower case string</td>
</tr>
<tr>
<td>age</td>
<td>Integer value giving age of customer</td>
</tr>
<tr>
<td>registration_date</td>
<td>Date of first registration with company (YYYY-MM-DD)</td>
</tr>
</tbody>
</table>
<p>Write a query to ensure that the <code>customer</code> table matches the description provided. </p>

In [1]:
%%sql 
postgresql:///insurance

--The submission accepts the query in the CTE as an answer. 
--I updated and altered the tables IRL just for fun.
--The result tables are limited after submission for readibility purposes.

WITH cte1 AS(
SELECT customer_id,
    LOWER(location) as location,
    LEFT(age, 2)::INTEGER as age,
    registration_date::DATE as registration_date
FROM customer
ORDER BY customer_id)

UPDATE customer AS c
SET customer_id = cte1.customer_id,
    location = cte1.location,
    age = cte1.age,
    registration_date = cte1.registration_date
FROM cte1
WHERE c.customer_id = cte1.customer_id;

ALTER TABLE customer
ADD CONSTRAINT uniqueness UNIQUE (customer_id),
ALTER COLUMN age SET DATA TYPE INTEGER USING age::INTEGER;

1500 rows affected.
Done.


[]

In [3]:
%%sql 
postgresql:///insurance
SELECT * FROM customer ORDER BY customer_id LIMIT 15

15 rows affected.


customer_id,location,age,registration_date
1,arkansas,32,2018-10-22
2,texas,23,2019-03-03
3,illinois,59,2018-04-12
4,pennsylvania,39,2018-09-05
5,new york,71,2019-01-29
6,arizona,85,2021-07-20
7,iowa,34,2019-07-05
8,alabama,59,2020-09-10
9,california,50,2020-07-20
10,new jersey,60,2020-06-02


## Task 2
<p>You have been informed that there has been a problem in the data entry. Some people who bought a new policy had their purchase type set to NULL instead of New.</p>
<p>Return the corrected <code>purchase</code> table.</p>

In [5]:
%%sql 
postgresql:///insurance

UPDATE purchase
SET purchase_type = 'New'
WHERE purchase_type IS NULL;

SELECT * FROM purchase LIMIT 15

0 rows affected.
15 rows affected.


purchase_id,purchase_type
0c16ed33,New
8530bd7b,Upgrade
1697a9d8,Upgrade
5439a130,Upgrade
c57c369b,Upgrade
d9d57db0,New
3d8ff616,Renewal
39044c72,Upgrade
b0a4aaeb,Upgrade
a5debeaf,Renewal


## Task 3
<p>The sales team wants to run a promotion on upgrades to international travel insurance policies. They only want to send this promotion to customers who have an active, US policy type.</p>
<p><img src="https://assets.datacamp.com/production/project_1642/img/schema.png" alt="database schema"></p>
<p>Write a query to provide the customer ID and start date for eligible customers. </p>

In [7]:
%%sql 
postgresql:///insurance  

SELECT customer_id, start_date
FROM policy as p
LEFT JOIN sales as s
ON p.policy_id = s.policy_id
WHERE policy_type = 'US' AND active = True
LIMIT 15

15 rows affected.


customer_id,start_date
6,2022-10-09
11,2022-11-14
18,2022-12-29
24,2022-06-11
26,2022-09-02
29,2022-12-14
30,2022-08-06
40,2022-08-14
45,2023-03-02
49,2023-03-25


## Task 4
<p>After the promotion has been sent, the sales team will need to monitor the number of active policy holders by policy type who purchased an upgrade.  </p>
<p><img src="https://assets.datacamp.com/production/project_1642/img/schema.png" alt="database schema"></p>
<p>Write a query that returns the data for the sales team to monitor. Your output should include <code>policy_type</code> and <code>number_active</code> columns.</p>

In [9]:
%%sql 
postgresql:///insurance

SELECT policy_type, COUNT(active) as number_active
FROM policy as p
LEFT JOIN sales as s
ON p.policy_id = s.policy_id
LEFT JOIN purchase as pr
ON s.purchase_id = pr.purchase_id
WHERE p.active = True AND purchase_type = 'Upgrade'
GROUP BY policy_type

2 rows affected.


policy_type,number_active
US,129
International,54
